In [1]:
from content_tree import *
import requests
import json
import pickle
import time

In [2]:
new_tree = ContentTree()

# copy some md files into this test folder so that you can test only one or a few md files!
md_directory = "./md_files"
print(f"Building tree from: {md_directory}")
new_tree.build_textbook_tree(md_directory, max_level=4)

# Rename repeating headers to make them unique
new_tree.rename_repeating_headers()
new_nodes = new_tree.tree_node_iterator()
print(len(new_nodes))


Building tree from: ./md_files
1218


In [3]:
# Check the file loading
with open('genchem_question_full.pkl', 'rb') as f:
    old_tree = pickle.load(f)
    print(old_tree.root.child_nodes[1].child_nodes[1].header_embedding)

llm_type = "openai"
llm_model = "gpt-4o" # other openai models: gpt-4o, gpt-4o-mini
llm_api_url = ""

#tree.process_tree_content()

old_nodes = old_tree.tree_node_iterator()
print(len(old_nodes))

[0.00471669 0.00788105 0.00105888 ... 0.01151251 0.00880121 0.01952702]
1218


In [4]:
for i in range(len(old_nodes)):
    o_node = old_nodes[i]
    n_node = new_nodes[i]
    print(o_node.header)
    print(n_node.header)
    n_node.summary = o_node.summary
    n_node.knowledge_list = o_node.knowledge_list
    n_node.keywords = o_node.keywords
    n_node.content_chunks = o_node.content_chunks
    n_node.sentences = o_node.sentences
    n_node.questions = o_node.questions
        
    # Embedding attributes (initialized when generated)
    n_node.header_embedding = o_node.header_embedding
    n_node.summary_embedding = o_node.summary_embedding
    n_node.chunk_embeddings = o_node.chunk_embeddings
    n_node.sentence_embeddings = o_node.sentence_embeddings
    n_node.question_embeddings = o_node.question_embeddings

Root
Root
Preface
Preface
Preface - About OpenStax
Preface - About OpenStax
Preface - About OpenStax resources Customization
Preface - About OpenStax resources Customization
Preface - Errata
Preface - Errata
Preface - Format
Preface - Format
Preface - About Chemistry 2e
Preface - About Chemistry 2e
Preface - Coverage and scope
Preface - Coverage and scope
Preface - Changes to the second edition
Preface - Changes to the second edition
Preface - Pedagogical foundation and features
Preface - Pedagogical foundation and features
Preface - Comprehensive art program
Preface - Comprehensive art program
Preface - Interactives that engage
Preface - Interactives that engage
Preface - Assessments that reinforce key concepts
Preface - Assessments that reinforce key concepts
Preface - Additional resources
Preface - Additional resources
Preface - Student and instructor resources
Preface - Student and instructor resources
Preface - Community Hubs
Preface - Community Hubs
Preface - Technology partners


In [5]:
counter = 0
for node in new_nodes:
    counter += 1
    if (node.header.find(" Key Terms") > -1 or node.header.find(" Summary") > -1 or node.header.find(" Exercises") > -1 or node.header.find("Root") > -1 or node.header.find("Preface") > -1 or node.header.find(" Key Equations") > -1):
        print("node.header", node.node_id, node.header)
        node.skip = True
    else:
        node.skip = False

node.header 1 Root
node.header 2 Preface
node.header 3 Preface - About OpenStax
node.header 4 Preface - About OpenStax resources Customization
node.header 5 Preface - Errata
node.header 6 Preface - Format
node.header 7 Preface - About Chemistry 2e
node.header 8 Preface - Coverage and scope
node.header 9 Preface - Changes to the second edition
node.header 10 Preface - Pedagogical foundation and features
node.header 11 Preface - Comprehensive art program
node.header 12 Preface - Interactives that engage
node.header 13 Preface - Assessments that reinforce key concepts
node.header 14 Preface - Additional resources
node.header 15 Preface - Student and instructor resources
node.header 16 Preface - Community Hubs
node.header 17 Preface - Technology partners
node.header 18 Preface - About the authors <br> Senior contributing authors
node.header 19 Preface - Paul Flowers, University of North Carolina at Pembroke
node.header 20 Preface - Klaus Theopold, University of Delaware
node.header 21 Pref

In [6]:
all_nodes = new_tree.tree_node_iterator()
content_nodes = [node for node in all_nodes if node.header_level > 0]

print(len(content_nodes))

1217


In [7]:
new_tree.process_tree_content(llm_type="openai", llm_model="gpt-4o", llm_api_url="")

COMPREHENSIVE CONTENT TREE PROCESSING
Processing 1171 content nodes...
LLM Provider: openai
LLM Model: gpt-4o
Embedding Model: text-embedding-3-large
Generate Embeddings: True
Create Inverse Index: True
--------------------------------------------------------------------------------

[1/1171] Processing node 2: Preface
Content length: 205 characters
  ⚠️ Node processing was skipped due to insufficient content.

[2/1171] Processing node 3: Preface - About OpenStax
Content length: 668 characters
  ⚠️ Node processing was skipped due to insufficient content.

[3/1171] Processing node 4: Preface - About OpenStax resources Customization
Content length: 944 characters
  ⚠️ Node processing was skipped due to insufficient content.

[4/1171] Processing node 5: Preface - Errata
Content length: 535 characters
  ⚠️ Node processing was skipped due to insufficient content.

[5/1171] Processing node 6: Preface - Format
Content length: 107 characters
  ⚠️ Node processing was skipped due to insufficient

In [8]:
# with open('genchem_question_full.pkl','wb') as f:
#     pickle.dump(new_tree, f)

with open('genchem_question_full.pkl','rb') as f:
    new_tree = pickle.load(f)

new_nodes = new_tree.tree_node_iterator()
print(len(new_nodes))

1218


In [9]:
for node in new_nodes:
    if (node.node_id > 28 and node.node_id < 2000):
        print(node.node_id, node.header)

29 The Domains of Chemistry
30 Chapter 1 1.2 Phases and Classification of Matter
31 Classifying Matter
32 Atoms and Molecules
33 Chapter 1 1.2 Phases and Classification of Matter Chemistry in Everyday Life
34 Chapter 1 1.2 Phases and Classification of Matter Chemistry in Everyday Life
35 Chapter 1 1.3 Physical and Chemical Properties
36 Chapter 1 1.3 Physical and Chemical Properties Chemistry in Everyday Life
37 Chapter 1 1.4 Measurements
38 SI Base Units
39 Length
40 Mass
41 Temperature
42 Time
43 Derived SI Units
44 Volume
45 Density
46 Example 1.1
47 Example 1.2
48 Chapter 1 1.5 Measurement Uncertainty, Accuracy, and Precision
49 Significant Figures in Measurement
50 Significant Figures in Calculations
51 Example 1.3
52 Example 1.4
53 Example 1.5
54 Example 1.6
55 Example 1.7
56 Accuracy and Precision
57 Chapter 1 1.6 Mathematical Treatment of Measurement Results
58 Conversion Factors and Dimensional Analysis
59 Example 1.8
60 Example 1.9
61 Example 1.10
62 Conversion of Temperature

In [10]:
# Test query with search analysis
#query = "What are the phases of matter?"
query = "What is the major elements of earth crust and air? What is the weight percentage of oxygen  on earth?"
print(f"\nAnalyzing query: {query}")

# Step 1: Show search results
search_results = new_tree.enhanced_search(query, max_results=5)
print(f"\nSearch results (top 5):")

all_nodes = new_tree.tree_node_iterator()
node_map = {node.node_id: node for node in all_nodes}

for i, (node_id, score) in enumerate(search_results, 1):
    if node_id in node_map:
        node = node_map[node_id]
        print(f"  {i}. [Node {node_id}] {node.header}")
        print(f"     Score: {score:.4f}")
        print(f"     Content preview: {node.content_text[:100]}...")
        print()

# Step 2: Show RAG answer
print(f"{'='*60}")
print("RAG Answer:")
print(f"{'='*60}")

llm_type = "openai"
llm_model = "gpt-4o" # other openai models: gpt-4o, gpt-4o-mini
llm_api_url = ""

answer, meta_data = new_tree.rag_query(query,llm_type=llm_type, llm_model=llm_model, llm_api_url=llm_api_url, top_k=5)
print(answer)

print("\n" + "="*80)
print("SEARCH ANALYSIS COMPLETE!")
print("="*80)


Analyzing query: What is the major elements of earth crust and air? What is the weight percentage of oxygen  on earth?
Generating embeddings for 1 texts in 1 batches...

Search results (top 5):
  1. [Node 31] Classifying Matter
     Score: 0.3675
     Content preview: Matter can be classified into several categories. Two broad categories are mixtures and pure substan...

  2. [Node 990] Chapter 18 18.9 Occurrence, Preparation, and Compounds of Oxygen
     Score: 0.3453
     Content preview: By the end of this section, you will be able to:

- Describe the properties, preparation, and compou...

  3. [Node 210] Oxidation-Reduction Reactions
     Score: 0.2809
     Content preview: Earth's atmosphere contains about $20 \%$ molecular oxygen, $\mathrm{O}_{2}$, a chemically reactive ...

  4. [Node 451] Chapter 9 9.1 Gas Pressure How Sciences Interconnect
     Score: 0.2626
     Content preview: **Meteorology, Climatology, and Atmospheric Science**

Throughout the ages, people have observed

In [12]:
questions = {"Classify Matters": [("How can you classify matters into a few simple categories?", 31),
    ("What is the major elements of earth crust and air? What is the weight percentage of oxygen  on earth?",31),
                                 ("Heterogeneous mixture example on a dinning table",31)],
"2.4 Chemical Formulas":[
("How does titanium dioxide provide protection from the sun?", 93),
("What is the difference between an empirical formula and a molecular formula?",[162,93,153, 158,94]),
("What different entities do a subscript following a symbol and a number in front represent?",93),
("What are some examples of structural isomers?",[1096, 95, 1090, 1064, 1065]),
("What are all the different types of isomers?",[1064,1096,95,1065])],
"Formation of Covalent Bonds":[
("In what cases would a covalent bond be formed instead of an ionic bond?",345),
("Could you please summarize the different types of covalent bonds?",345),
("What determines whether a bond is nonpolar or polar covalent?",345),
("Why is breaking bonds an endothermic process while forming chemical bonds is an exothermic process?",345)],
"Electronegativity":[
("What is the difference between electronegativity and electron affinity?",347),
("Is it possible to measure electronegativity?",347)],
"Electronegativity versus Electron Affinity":[
("What is the difference between electronegativity and electron affinity?",348),
("Is it possible to measure electronegativity?",348)],
"Electronegativity and Bond Type":[
("What are some common polyatomic ions I should know?",350),
("What is the best method for determining whether a bond is considered ionic or covalent?",350),
("How come some compounds contain both covalent and ionic bonds?",350)],
"10.6 Lattice Structures in Crystalline Solids":[
("Why would the coordination number for polonium metal be six?",554),
("Is polonium found in nature?",  1139),
#("If polonium is found in nature, why would such an inefficient structure form in nature?", 551),
#("If there are isomorphous metals, are there amorphous metals?",551),//
#("If so, what are the properties of amorphous metals?", 551),//
("Could you please summarize the four major types of unit cells and their differences?", [555,556]),
#("How can you have one eight of an atom at each of the corners in a body-centered cubic solid?", 555),
#("Why do atoms being on identical lattice points correspond to them having an identical environment?", 556),
("Could you please clarify why CCP and FCC arrangements are considered identical?", 556),
#("Why would some metals crystallize in a certain structure over another?",555),
("Could you please summarize the seven different types of lattice systems?", 557),
("What factors influence how the structures of ionic crystals form?",558),
("Could you please explain the difference between tetrahedral and octahedral holes?", 558),
("Why would some of the tetrahedral holes remain vacant in compounds with maximum cation: maximum anion ratio less than 2:1?",558),
("Does whether the tetrahedral or octahedral holes are filled or empty change any properties of the crystalline structures?",558),
#?("What can cause different types of ionic compounds to crystallize in the same structure?",561),
("Why is CaCI not a BCC structure?",[556,561]),
("In Figure 10.59, isn’t there one Cs per 8 CI so how can it be a 1:1 ratio?", 561),
("What is the difference between the structure of NaCI and CsCI?", 561),
#?("How come both ZnS and NaCI form an FCC structure but the two diagrams for them look very different?", [556, 561]),
("Both ZnS and NaCI form an FCC structure, can you tell me the difference in their unit cell?", [556, 561]),
#?("How was the Bragg equation derived?", 564),
("In the definition for diffraction, what does 'when it encounters a physical barrier whose dimensions are comparable to those of the wavelength of light' mean?", 564),
#?("How can the dimensions of a physical barrier be similar to the dimensions of the wavelength of light?", 564),
#?("Wouldn’t other types of interference be possible (not only constructive and destructive)?",564),
("What does first order diffraction mean?", [564, 565])],
# "20.1 Hydrocarbons":[
# ("What are organic compounds?", 1087),
# ("What causes differences between organic compounds?",1087),
# ("Which elements do organic compounds contain?", 1087),
# ("If plastic is made up of a hydrocarbon, why can’t it decompose?", 1087),
# ("What is the difference between alkanes and hydrocarbons?",1088),
# ("In what cases would you use a condensed formula instead of a Lewis structure?",1088),
# ("How do chemists simplify the drawings of larger structures?",1088),
# ("Why is specifically carbon emitted from the skeletal structures and not some other kind of element?", 1088),
# ("In the skeletal structures, how do you know where to place the carbons?", 1089),
# ("In skeletal structures, how do you know when to place CH2 instead of CH or CH3 if you are trying to expand the structure?",[1089,1090]),
# ("What kinds of intermolecular attractions or forces are there?", 1090),
# ("Why do greater numbers of atoms in the molecules lead to stronger intermolecular attractions?", 1090),
# ("In the table of “Properties of Some Alkanes”, why do more hydrogens and carbons in the molecule lead to more structural isomers being possible?", 1090),
# ("How can you tell if two Lewis structures that look different actually represent the same molecule?", 1090),
# ("What does nomenclature mean?", 1091),
# ("What are the two naming rules for alkanes?", 1091),
# ("How do you obtain the name of an alkyl group?", 1092),
# ("How are a number of substituents of the same type indicated?",1091),
# ("How do you name an electronegative substituent?",1091),
# ("Could you please provide a concise table summarizing how naming alkanes works?",1091),
# ("Why do alkyl groups not exist as stable independent entities?",1091),
# ("What is the difference between alkanes and alkyl groups?",[1091, 1093]),
# ("Are alkanes good fuels?", [1092, 1093]),
# ("How does the process of fractional distillation work?",1093),
# ("Could you please summarize the typical reactions of alkanes?", 1093),
# ("What are some common functional groups and what exactly are they?", 1095),
# ("What is the difference between alkenes and alkanes?",1096),
# ("Could you please summarize all the different kinds of isomers and their properties?", 1096),
# ("What does moiety mean?",1093),
# ("What are the different properties of alkynes?", 1099)],
"Free Energy": [
("What is entropy?",[852,853]),
("Could you please tell me who introduced Gibbs free energy?",863),
("What does it mean for something to be a state function?", 864),
("Could you please clarify the process for deriving the Gibbs free energy equation?",863),
("Why does an exothermic process involve a decrease in entropy?",865),
("What is thermodynamic reversibility?",864),
("What is the difference between standard free energy of formation and Gibbs free energy?",[863,864,865]),
("What does the sigma symbol stand for in the calculation of free energy change?", 865),
("For the standard free energy of formation calculation, why must absolute value be used?",865),
("What does it mean for something to be thermodynamically unfavorable?",[863,864]),
("How do coupled reactions work for calculating free energy change of a chemical reaction?", [868,869]),
("What does a negative Gibbs free energy change indicate for a chemical reaction?",863),
("Why is equilibrium established when the system’s free energy is minimized?",873),
("Why is there an log of reaction quotient (ln Q) present in the equation for Gibbs free energy under nonstandard conditions?", 709),
("Why would the estimation of phase-transition temperature using the enthalpy and entropy changes closer to boiling point be more accurate than results using the standard thermodynamic properties?",[869,870]),
("The form of the equation for Gibbs free energy under nonstandard conditions provides a useful link for which two thermodynamic properties?",863)
]}

In [13]:
# Question, node_ids, headers
q_list = []
for key in questions.keys():
    q_list += questions[key]
print(q_list)
print(len(q_list))

[('How can you classify matters into a few simple categories?', 31), ('What is the major elements of earth crust and air? What is the weight percentage of oxygen  on earth?', 31), ('Heterogeneous mixture example on a dinning table', 31), ('How does titanium dioxide provide protection from the sun?', 93), ('What is the difference between an empirical formula and a molecular formula?', [162, 93, 153, 158, 94]), ('What different entities do a subscript following a symbol and a number in front represent?', 93), ('What are some examples of structural isomers?', [1096, 95, 1090, 1064, 1065]), ('What are all the different types of isomers?', [1064, 1096, 95, 1065]), ('In what cases would a covalent bond be formed instead of an ionic bond?', 345), ('Could you please summarize the different types of covalent bonds?', 345), ('What determines whether a bond is nonpolar or polar covalent?', 345), ('Why is breaking bonds an endothermic process while forming chemical bonds is an exothermic process?'

In [14]:
import pandas as pd

q_data = []
for q in q_list:
    question = q[0]
    node_ids = q[1]
    # Check 
    if (type(node_ids) is list):
        headers = [new_nodes[node_id-1].header for node_id in node_ids]
    else:
        headers = [new_nodes[node_ids-1].header]
        node_ids = [node_ids]
    q_data.append([question, node_ids, headers])

question_df = pd.DataFrame(q_data, columns=['Question', 'Node IDs', 'Markdown Headers'])
question_df.head(100)

,Question,Node IDs,Markdown Headers
0,How can you classify matters into a few simple...,[31],[Classifying Matter]
1,What is the major elements of earth crust and ...,[31],[Classifying Matter]
2,Heterogeneous mixture example on a dinning table,[31],[Classifying Matter]
3,How does titanium dioxide provide protection f...,[93],[Chapter 2 2.4 Chemical Formulas]
4,What is the difference between an empirical fo...,"[162, 93, 153, 158, 94]","[Derivation of Molecular Formulas, Chapter 2 2..."
5,What different entities do a subscript followi...,[93],[Chapter 2 2.4 Chemical Formulas]
6,What are some examples of structural isomers?,"[1096, 95, 1090, 1064, 1065]","[Isomers of Alkenes, Chapter 2 2.4 Chemical Fo..."
7,What are all the different types of isomers?,"[1064, 1096, 95, 1065]","[Isomerism in Complexes, Isomers of Alkenes, C..."
8,In what cases would a covalent bond be formed ...,[345],[Formation of Covalent Bonds]
9,Could you please summarize the different types...,[345],[Formation of Covalent Bonds]


In [15]:
import pickle

with open("test_data_df.pkl","wb") as f:
    pickle.dump(question_df, f)

In [16]:
# Load test data set
import pickle

with open("test_data_df.pkl","rb") as f:
    test_data_df = pickle.load(f)

In [18]:
# Test customized weights for scoring
from parameters import (
    DEFAULT_PARAMETERS, 
    SEMANTIC_FOCUSED_PARAMETERS,
    LEXICAL_FOCUSED_PARAMETERS,
    create_custom_parameters
)

custom_config = create_custom_parameters(
    # Semantic similarity weights
    semantic_header=0.2,
    semantic_summary=0.3,
    semantic_content=0.1,
    semantic_chunks=0.2,
    semantic_sentences=0.1,
    semantic_questions=0.25,
    # N-gram weights  
    ngram_monogram=0.5,
    ngram_bigram=2.0,
    ngram_trigram=4.5,
    # Combined search weights
    combined_semantic=0.7,
    combined_lexical=0.3
)

top1_acc = 0
top2_acc = 0
top3_acc = 0
total_top2 = 0
total_top3 = 0
total = len(test_data_df)
for i in range(len(test_data_df)):
    total_top2 += 1
    total_top3 += 1
    row = test_data_df.iloc[i]
    query = row['Question']
    node_ids = row['Node IDs']
    header = row['Markdown Headers']
    time1 = time.time()
    if len(node_ids) > 1:
        total_top2 += 1
        total_top3 += 1
    if len(node_ids) > 2:
        total_top3 += 1
    #answer, meta_data = new_tree.rag_query(query, top_k=5, custom_params=custom_config)
    # Step 1: Show search results
    search_hits = new_tree.enhanced_search(query, max_results=5, parameters=custom_config)

    #print("-------------------------------------------------")
    #print("Time = ", time.time()-time1)
    #print("Question", query)
    # print("Answer",answer)
    # search_hits = meta_data.get("search_results",[])
    #print("Node IDs = ", node_ids)
    #print("Headers = ", header)
    #print("Content hits = ",search_hits)
    n_ids = [x[0] for x in search_hits]
    for j in range(len(n_ids)):
        if (j == 0 and n_ids[j] in node_ids):
            top1_acc += 1
            top2_acc += 1
            top3_acc += 1
        if (j == 0 and n_ids[0] not in node_ids):
            print("-------------------------------------------------")
            print("Content IDs", node_ids)
            print("Search IDs ", search_hits)
            print("Question:", query)
            print("Headers:", header)
        if (j == 1 and n_ids[j] in node_ids):
            top2_acc += 1
            top3_acc += 1        
        if (j == 2 and n_ids[j] in node_ids):
            top3_acc += 1 
print("topn_acc = ", top1_acc, top2_acc, top3_acc)
print("topn_percentages = ", top1_acc/total, top2_acc/total_top2, top3_acc/total_top3)

Generating embeddings for 1 texts in 1 batches...
Generating embeddings for 1 texts in 1 batches...
Generating embeddings for 1 texts in 1 batches...
Generating embeddings for 1 texts in 1 batches...
Generating embeddings for 1 texts in 1 batches...
Generating embeddings for 1 texts in 1 batches...
Generating embeddings for 1 texts in 1 batches...
Generating embeddings for 1 texts in 1 batches...
Generating embeddings for 1 texts in 1 batches...
Generating embeddings for 1 texts in 1 batches...
-------------------------------------------------
Content IDs [345]
Search IDs  [(419, 0.38021343074187913), (346, 0.3636657490854272), (344, 0.35844281645982407), (409, 0.3574446044229497), (350, 0.3573755307754142)]
Question: Could you please summarize the different types of covalent bonds?
Headers: ['Formation of Covalent Bonds']
Generating embeddings for 1 texts in 1 batches...
-------------------------------------------------
Content IDs [345]
Search IDs  [(347, 0.6664199057826755), (350, 0